# ADM-HW3: GROUP #14

#### Xavier Del Giudice, Alessio Iacono, Geraldine Maurer


| STUDENT |   ID    |                 E-mail                  |
| :-: |:-------:|:---------------------------------------:|
| Xavier Del Giudice | 1967219 | delgiudice.1967219@studenti.uniroma1.it |
| Alessio Iacono | 1870276 |   iacono.1870276@studenti.uniroma1.it   |
| Geraldine Maurer | 1996887 |           gmaurer0@gmail.com            |

## Import Packages 

In [1]:
%pip install unidecode geopy plotly dash aiofiles aiohttp nltk ipywidgets requests bs4 pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import heapq
import nltk
import os
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from unidecode import unidecode
import string
import unicodedata
from IPython.display import display, clear_output
import pandas as pd
from collections import defaultdict
import pickle
import ipywidgets
import ipywidgets as ipw
from itertools import chain
from functions import utils, engine
from IPython.display import display

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# 1. Data Collection

In [36]:
base_url = "https://guide.michelin.com/en/it/restaurants"
pages = 100
utils.scrape_michelin_restaurants(base_url, pages)

Fetching https://guide.michelin.com/en/it/restaurants/page/1...
Fetching https://guide.michelin.com/en/it/restaurants/page/2...
Fetching https://guide.michelin.com/en/it/restaurants/page/3...
Fetching https://guide.michelin.com/en/it/restaurants/page/4...
Fetching https://guide.michelin.com/en/it/restaurants/page/5...
Fetching https://guide.michelin.com/en/it/restaurants/page/6...
Fetching https://guide.michelin.com/en/it/restaurants/page/7...
Fetching https://guide.michelin.com/en/it/restaurants/page/8...
Fetching https://guide.michelin.com/en/it/restaurants/page/9...
Fetching https://guide.michelin.com/en/it/restaurants/page/10...
Fetching https://guide.michelin.com/en/it/restaurants/page/11...
Fetching https://guide.michelin.com/en/it/restaurants/page/12...
Fetching https://guide.michelin.com/en/it/restaurants/page/13...
Fetching https://guide.michelin.com/en/it/restaurants/page/14...
Fetching https://guide.michelin.com/en/it/restaurants/page/15...
Fetching https://guide.michelin.co

In [37]:
# Read and visualize the lines of the file
with open("michelin_restaurant_urls.txt", "r") as file:
    urls = file.readlines()
# Remove the empty spaces
urls = [url.strip() for url in urls]
print(f"Total number of URL collected: {len(urls)}")

Total number of URL collected: 3381


In [38]:
# Remove the duplicates
unique_urls = list(set(urls))
print(f"Number of unique URL: {len(unique_urls)}")
# Filter the URLs that contains "/restaurant/"
restaurant_urls = [url for url in unique_urls if "/restaurant/" in url]
print(f"Number of valid URLs: {len(restaurant_urls)}")

Number of unique URL: 1991
Number of valid URLs: 1981


In [39]:
with open("michelin_restaurant_urls_cleaned.txt", "w") as file:
    for url in restaurant_urls:
        file.write(url + "\n")

print("Cleaned file saved as 'michelin_restaurant_urls_cleaned.txt'")

Cleaned file saved as 'michelin_restaurant_urls_cleaned.txt'


In [40]:
# Legge e visualizza le prime righe del file
with open("michelin_restaurant_urls_cleaned.txt", "r") as file:
    urls = file.readlines()
    
# Rimuove gli spazi vuoti e visualizza un esempio di URL raccolti
urls = [url.strip() for url in urls]
print(f"Total number of URL collected: {len(urls)}")

Total number of URL collected: 1981


In [ ]:
# Path with all the URLs
file_path = "C:/Users/xavie/Downloads/michelin_restaurant_urls_cleaned.txt"

functions.process_urls(file_path)

Saved HTML of https://guide.michelin.com/en/piemonte/san-marzano-oliveto/restaurant/del-belbo-da-bardon in page_1\html_del-belbo-da-bardon.txt
Saved HTML of https://guide.michelin.com/en/emilia-romagna/santarcangelo-di-romagna/restaurant/osteria-la-sangiovesa in page_1\html_osteria-la-sangiovesa.txt
Saved HTML of https://guide.michelin.com/en/emilia-romagna/rimini/restaurant/trattoria-da-lucio in page_1\html_trattoria-da-lucio.txt
Saved HTML of https://guide.michelin.com/en/puglia/peschici/restaurant/porta-di-basso in page_1\html_porta-di-basso.txt
Saved HTML of https://guide.michelin.com/en/veneto/padova/restaurant/stefano-mocellin-al-padovanino in page_1\html_stefano-mocellin-al-padovanino.txt
Saved HTML of https://guide.michelin.com/en/sardegna/portoscuso/restaurant/sa-musciara in page_1\html_sa-musciara.txt
Saved HTML of https://guide.michelin.com/en/lombardia/rho/restaurant/mezzolitro-vini-e-cucina in page_1\html_mezzolitro-vini-e-cucina.txt
Saved HTML of https://guide.michelin.co

In [36]:
import glob
# Directory di base contenente tutte le cartelle di download
base_dir = 'C:/Users/xavie/Documents/ADM_HW3'

# Lista per memorizzare i dati di ogni ristorante
restaurants_data = []

# Iterazione su tutte le cartelle page_1, page_2, ..., page_100
for page_num in range(1, 101):
    folder_path = os.path.join(base_dir, f"page_{page_num}")
    
    # Trova tutti i file di testo che iniziano con "html_" nella cartella corrente
    html_files = glob.glob(os.path.join(folder_path, "html_*.txt"))
    print(html_files)
    
    for file_path in html_files:
        restaurant_info = functions.extract_restaurant_info(file_path)
        restaurants_data.append(restaurant_info)

# Creazione del DataFrame Pandas
df = pd.DataFrame(restaurants_data)

output_file_path = "C:/Users/xavie/Documents/ADM_HW3/restaurants_data_test1.tsv"
df.to_csv(output_file_path, sep='\t', index=False)
print(f"Data has been saved in {output_file_path}")

Create TXT with urls of each restaurant page (just let it run, usually 2 minutes to finish)

### Integriamo lo scrape dell url con il pezzo sotto

In [3]:
import requests
from bs4 import BeautifulSoup

# URL di partenza
base_url = 'https://guide.michelin.com/en/it/restaurants/page/'

def scrape_restaurant_links():
    print("I'm starting to Scrape!")
    page = 1
    all_links = []

    while True:
        # Costruisci l'URL della pagina corrente
        url = f"{base_url}{page}"
        response = requests.get(url)

        # Verifica che la richiesta sia andata a buon fine
        if response.status_code != 200:
            print(f"Errore nel caricamento della pagina {page}")
            break

        # Parsing della pagina HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Trova tutti i div con la classe specificata
        for class1_div in soup.select("div.card__menu-content.card__menu-content--flex.js-match-height-content"):
            # Cerca il tag <h3> con classe specificata e il tag <a> figlio
            h3 = class1_div.select_one("h3.card__menu-content--title.pl-text.pl-big.js-match-height-title a")
            if h3:
                link = h3.get("href")
                full_link = "https://guide.michelin.com" + link if link else None
                if full_link:
                    all_links.append(full_link)

        # Trova la sezione di paginazione
        pagination_lis = soup.select("div.js-restaurant__bottom-pagination ul li")

        # Trova l'elemento <li> con la classe "active"
        active_index = None
        for i, li in enumerate(pagination_lis):
            if li.select_one("a.active"):
                active_index = i
                break

        # Se c'è una pagina successiva, incrementa il numero di pagina
        if active_index is not None and active_index + 1 < len(pagination_lis):
            next_page = pagination_lis[active_index + 1].select_one("a")
            if next_page and next_page.get("href"):
                page += 1
            else:
                break
        else:
            break

    # Salva tutti i link dei ristoranti in un file
    with open("soupUrls.txt", "w") as file:
        for link in all_links:
            file.write(link + "\n")

    print(f"Scraping completed. {len(all_links)} link saved in soupUrls.txt.")

# Avvia lo scraping
scrape_restaurant_links()

Download each HTML page using .txt file just created.

In [4]:
#Scaricare html content da url

import aiohttp
import asyncio
import aiofiles
import os

CONCURRENT_REQUESTS = 20  # Lowered to reduce load on the server

async def load_urls(file_path):
    async with aiofiles.open(file_path, 'r') as f:
        urls = [line.strip() for line in await f.readlines()]
    return urls

async def download_url(session, url, output_dir):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36',
        'Referer': 'https://guide.michelin.com/',
        'Accept-Language': 'en-US,en;q=0.9',
    }
    try:
        async with session.get(url, headers=headers) as response:
            if response.status == 200:
                content = await response.text()
                filename = f"{output_dir}/{hash(url)}.html" ### MODIFICARE CON IL NOME DEL RISTORANTE .TXT ###
                async with aiofiles.open(filename, 'w') as f:
                    await f.write(content)
                print(f"Downloaded: {url}")
            else:
                print(f"Failed to download {url}: Status {response.status}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")
 ### OGNI 20 DOWNLOAD CREA UNA NUOVA CARTELLA page_* DENTRO IL PATH : OUTPUT DIR ###
async def download_all(urls, output_dir):
    connector = aiohttp.TCPConnector(limit=CONCURRENT_REQUESTS)
    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = [download_url(session, url, output_dir) for url in urls]
        await asyncio.gather(*tasks)

if __name__ == "__main__":
    file_path = 'soupUrls.txt'
    output_dir = 'downloads'
    os.makedirs(output_dir, exist_ok=True)

    urls = await load_urls(file_path)
    await download_all(urls, output_dir)

Scrape each HTML page and create dataframe from data.

In [5]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup

# Directory containing the downloaded HTML files
output_dir = 'downloads'

# List to store restaurant data
restaurants_data = []

# Function to extract restaurant information from HTML
def extract_restaurant_info(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

        # Extract information using CSS selectors or HTML structure of the page
        restaurant_info = {}

        # Ricerca div contenente info principali
        restaurantDetailsDiv = soup.find("div", class_="restaurant-details__components")

        # Ottiene tutte le row contenenti: Nome del ristorante (row1),
        # Indirizzo, prezzo e tipo cucina (row2), row 3 da scartare
        mainInfo = restaurantDetailsDiv.select("div.data-sheet > div.row")

        if mainInfo[0]:
            restaurant_info['restaurantName'] = mainInfo[0].find("h1", class_="data-sheet__title").text
        if mainInfo[1]:
            indirizzo_price = mainInfo[1].select("div.data-sheet__block > div.data-sheet__block--text")

            # Splitta la stringa contenente indirizzo, citta, CAP e nazione
            indirizzoList = indirizzo_price[0].text.strip().split(",")

            # Seleziona gli ultimi tre e li assegna a country, postalCode e city, tutto il resto verrà assegnato ad indirizzo
            restaurant_info['city'] = indirizzoList[-3]
            restaurant_info['postalCode'] = indirizzoList[-2]
            restaurant_info['country'] = indirizzoList[-1]
            restaurant_info['address'] = " ".join(indirizzoList[:-3]).strip().replace("\n", "") # Unisce tutti gli elementi precedenti agli ultimi tre

            # Split della riga contenente price e cuisineType info
            restaurant_info['priceRange'], restaurant_info['cuisineType'] = indirizzo_price[1].text.strip().split("·")

            restaurant_info['priceRange'] = restaurant_info['priceRange'].strip()
            # Possibili multiple cuisineType, dividi in lista
            restaurant_info['cuisineType'] = restaurant_info['cuisineType'].strip().split(",")

        # Description
        restaurant_info['description'] = soup.find("div", class_="data-sheet__description").text.strip().replace("\n", "")

        # Facilities and Services
        facilities = soup.select("div.restaurant-details__services ul li")
        restaurant_info['facilitiesServices'] = [s.text.strip() for s in facilities]

        # Accepted Credit Cards
        credit_cards = soup.select("div.list--card img")
        restaurant_info['creditCards'] = [re.search(r"(?<=\/)[a-z]*(?=-)", c.get("data-src"))[0] for c in credit_cards]

        # Phone Number
        spansDetails = restaurantDetailsDiv.select("section.section.section-main.section__text-componets.section__text-separator div.collapse__block-title div.d-flex span")
        restaurant_info['phoneNumber'] = spansDetails[0].text.strip()

        # URL
        restaurant_info['website'] = soup.find("meta", property="og:url")["content"]

    return restaurant_info

# Loop through all files in the directory and extract information
for filename in os.listdir(output_dir):
    if filename.endswith(".html"): ### SCORRI TUTTE LE CARTELLE E PRENDI PER OGNI CARTELLA FILE CHE TERMINANO PER .TXT ###
        print(filename)
        file_path = os.path.join(output_dir, filename)
        restaurant_info = extract_restaurant_info(file_path)
        restaurants_data.append(restaurant_info)

# Create a pandas DataFrame
df = pd.DataFrame(restaurants_data)

# Save the data to a CSV file
df.to_csv("restaurants_data.tsv", sep='\t', index=False)
print("Data saved to restaurants_data.csv")

# 2. Search Engine

## 2.0 Preprocessing

We collected all the useful defined functions that we used to run the Engine, in the specified file: ```engine.py```.
You can find it in the folder ````functions````... TODO

The first step is to preprocess the restaurant descriptions. For this, we use the custom-made function ```preprocessing```, and save all pre-processed documents in a list of documents ```preprocessed_docs```.

In [3]:
df = pd.read_csv('restaurants_data.tsv', sep = '\t')

In [4]:
df.head()

,restaurantName,city,postalCode,country,address,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,Arca,San Benedetto del Tronto,63074,Italy,viale Rinascimento 137,€€€,"Modern Cuisine, Cuisine from Abruzzo","In business for over 20 years, Arca is definit...","['Air conditioning', 'Restaurant offering vege...","['Amex', 'Mastercard', 'Visa']",+39 0735 488908,https://www.arcaristorante.it/
1,Cannavacciuolo Le Cattedrali Asti,Asti,14100,Italy,frazione Valleandona 1/b,€€€€,Creative,"Situated in the hills outside Asti, this moder...","['Air conditioning', 'Car park', 'Garden or pa...","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 0141 185 8888,https://www.lecattedrali.com/
2,Capriccio,Manerba del Garda,25080,Italy,piazza San Bernardo 6 località Montinelle,€€€,"Modern Cuisine, Seafood",This restaurant standing on the shores of Lake...,"['Air conditioning', 'Car park', 'Great view',...","['Amex', 'Maestrocard', 'Mastercard', 'Visa']",+39 0365 551124,https://ristorantecapriccio.it/
3,Cestello Firenze,Florence,50124,Italy,piazza di Cestello 8,€€€,"Seafood, Contemporary",Situated in the beautiful piazza overlooked by...,"['Air conditioning', 'Terrace']","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 055 264 5364,NaN
4,Corte Sconta,Venice,30122,Italy,calle del Pestrin sestiere Castello 3886,€€€,"Seafood, Venetian","From its flooring to its tables, this restaura...","['Air conditioning', 'Terrace']","['Mastercard', 'Visa']",+39 041 522 7024,https://www.cortescontave.com


In [3]:
preprocessed_docs = defaultdict(list) # initialize defaultdict to store preprocessed docs
for doc_id, doc in enumerate(df.description):
  preprocessed_docs[doc_id] = engine.preprocessing(doc) # preprocess doc at position doc_id

In [18]:
# Test description
text = '''After many years' experience in Michelin-starred restaurants, Luigi Tramontano and his wife Nicoletta
have opened their first restaurant in the chef's native Gargnano. Previously a pasta factory, the building has been converted
into an elegant, contemporary-style restaurant which has nonetheless retained its charming high ceilings.
The cuisine is inspired by regional traditions which are reinterpreted to create gourmet dishes,
all prepared with respect for the ingredients used and a strong focus on local produce.'''

# Test preprocessing on test description
print(engine.preprocessing(text))

['mani', 'year', 'experi', 'michelin', 'star', 'restaur', 'luigi', 'tramontano', 'wife', 'nicoletta', 'open', 'first', 'restaur', 'chef', 'nativ', 'gargnano', 'previous', 'pasta', 'factori', 'build', 'convert', 'eleg', 'contemporary', 'styl', 'restaur', 'nonetheless', 'retain', 'charm', 'high', 'ceil', 'cuisin', 'inspir', 'region', 'tradit', 'reinterpret', 'creat', 'gourmet', 'dish', 'prepar', 'respect', 'ingredi', 'use', 'strong', 'focu', 'local', 'produc']


## 2.1 Conjunctive Query

### 2.1.1 Create your Index!

In the following code cells, we preprocess all restaurant descriptions and 1. save unique tokens in a DataFrame ```vocabulary_df``` that maps terms to unique integer IDs, then 2. compute the inverted index for the documents.

In [4]:
# 1. Vocabulary File

# Retrieve the restaurants DataFrame
df = pd.read_csv('restaurants_data.tsv', sep='\t')

doc_tokens = [] # initialize list to store all tokens

# Find unique tokens
for doc in preprocessed_docs.values():
  doc_tokens.extend(doc)
  doc_tokens = list(set(doc_tokens)) # remove duplicates

vocabulary_dict = {term: i for i,term in enumerate(doc_tokens)} # dictionary of all vocabulary terms
vocabulary_df = pd.DataFrame({'term': vocabulary_dict.keys(), 'term_id': vocabulary_dict.values()}) # dataframe that maps terms to IDs

vocabulary_df.to_csv('vocabulary.csv', index=False) # save vocabulary dataframe in a csv file

In [5]:
# 2. Inverted Index

inverted_index = defaultdict(list) # initialize inverted_index dictionary

# Compute the inverted_index
for doc_id, row in enumerate(df.description):
  tokens = set(preprocessed_docs[doc_id]) # preprocessed description
  for token in tokens: # eliminate duplicates
    # Look up the term_id of the current term/token
    term_id = vocabulary_dict[token]
    # If the doc_id is not in the term_id's list in inverted_index, add it
    if doc_id not in inverted_index[term_id]:
      inverted_index[term_id].append(doc_id)

# Save the inverted_index dictionary to a file
with open("inverted_index.pkl", "wb") as file:
    pickle.dump(inverted_index, file)

Next, we allow the user to input a query. After clicking on search, the first search engine will be triggered to retrieve all restaurants that contain in their description the same terms as the query.

In [21]:
import ipywidgets as widgets
from IPython.display import display
# re-load inverted index in case it was modified somewhere
with open('inverted_index.pkl', 'rb') as file:
    inverted_index = pickle.load(file)

# Text input field for query
text_input = widgets.Text(
    value='',
    placeholder='Type your query',
    description='Query:',
    disabled=False
)

# Search button
search_button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='primary'
)

output = widgets.Output()

# Define a function to handle button press
def on_search_button_clicked(b):
    with output:
        output.clear_output()  # clear previous output if there are any
        query = text_input.value
        if query.strip():  # Check if there's an input
            display(engine.find_restaurants(query, vocabulary_df, inverted_index, df)) # display query results
        else:
            print("Please enter something to search for.")

# Link the function to the button
search_button.on_click(on_search_button_clicked)

# Display the widgets
display(text_input, search_button, output)

Text(value='', description='Query:', placeholder='Type your query')

Button(button_style='primary', description='Search', style=ButtonStyle())

Output()

In [13]:
# with query: "modern seasonal cuisine" we obtained this result
output

Output()

## 2.2 Ranked Search Engine with TF-IDF and Cosine Similarity

### 2.2.1 Inverted Index with TF-IDF Scores

In the following exercise, we will first compute the inverted index with TF-IDF scores using the custom-made function ```tf_idf``` and save the ```updated_inverted_index``` in a pickle file.

In [7]:
# Preliminary steps
n = len(preprocessed_docs)
updated_inverted_index = defaultdict(list) # initialize default dictionary to store the inverted_index values with TF-IDF scores
inverted_index_copy = inverted_index.copy() # Create a copy of the inverted_index to iterate over

# Compute updated_inverted_index
for term_id, docs in inverted_index_copy.items():
  tf_idf_scores = engine.tf_idf(int(term_id), inverted_index, preprocessed_docs, vocabulary_df, n)
  updated_inverted_index[term_id] = list(zip(docs, tf_idf_scores))

with open('updated_inverted_index.pkl', 'wb') as file:
    pickle.dump(updated_inverted_index, file)

Next, we retrieve from ```updated_inverted_index``` the TF-IDF scores related to documents, and memorize only the tuples (term, tf-idf) where tf-idf != 0 for each document in a pickle file.

In [8]:
# Compute the TF-IDF vectors of all documents and store them in a pickle file
doc_tf_idf_scores = defaultdict(list) # initialize dictionary to store non-zero TF-IDF scores for each document

for term_id, docs_scores in updated_inverted_index.items():
  for doc_id, tf_idf_score in docs_scores:
    if tf_idf_score != 0:
      doc_tf_idf_scores[doc_id].append((term_id,tf_idf_score))
  doc_tf_idf_scores[doc_id].sort(key=lambda x: x[0]) # sort the terms

with open('doc_tf_idf_scores.pkl', 'wb') as file:
    pickle.dump(doc_tf_idf_scores, file)

Finally, we enable the user to input a text query, and return the top-k ranked restaurants by cosine similarity.

In [24]:
# re-load inverted index in case it was modified somewhere
with open('inverted_index.pkl', 'rb') as file:
    inverted_index = pickle.load(file)

# Text input field for query
text_input = widgets.Text(
    value='',
    placeholder='Type your query',
    description='Query:',
    disabled=False
)

# Search button
search_button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='primary'
)

output = widgets.Output()

# Define a function to handle button press
def on_search_button_clicked(b):
    with output:
        output.clear_output()  # clear previous output if there are any
        query = text_input.value
        if query.strip():  # Check if there's an input
            k = 10
            display(engine.top_k_restaurants(query, inverted_index, vocabulary_dict, doc_tf_idf_scores, df, n, k)) # display query results
        else:
            print("Please enter something to search for.")

# Link the function to the button
search_button.on_click(on_search_button_clicked)

# Display the widgets
display(text_input, search_button, output)

Text(value='', description='Query:', placeholder='Type your query')

Button(button_style='primary', description='Search', style=ButtonStyle())

Output()

In [12]:
output

Output()

## 3. Define a New Score!

A user interface(UI) is used to consider the user's additive information, which allows the user to enter the fields they prefer to find the restaurant.
We will use this additional information to predicate some of the rows in our dataset.
Since all restaurants are filtered by text initially, we want to give lower weight to the similarity between the description and the input query.  
Rather we want to give higher weight to user input details, which are the key to finding restaurants to which a user can bring interest.
Specifically we prefer to give higher weight to the type of cuisine, if I were to search for a restaurant and I have a particular desire to eat that cuisine I would want those as the pirmiest search results. Sucessively we give slightly less weight to the services available and finally the least weight to the price, speaking of Michelin restaurants the price is not an issue
We can also find this behavior in very popular search engines, for example amazon search, prefers user-selected filters over the user's even detailed content in the search bar.

The results in the end turn out to be better, more accurate. We also have to consider that in the previous Engine based on the cosine similarity we are just considering the score based on the similarity between the input query and the description which is way larger than a possible query. Instead now we are building our own custom score where we give more importance to the details, infact we havebigger score for the ones that perfectly fit thee search. We are basically considernig more fileds so its obvious that the results are more accurated. But there is also to note that at the computational level this type of search is less performant since we have to initially load all the restaurants that respect the query input and then create a new score based on matching the user input and finally sort to select only the first k.

In [14]:
from functions.search_restaurant_ui import SearchRestaurantUI

In [15]:
search_ui = SearchRestaurantUI(df, vocabulary_df, inverted_index)
search_ui.display()

As the previous points, we can not visualize the UI just created. To let you understand what we created we are going to use an image of it.
But if you would like to use it, you can download the code and run it in local.
In the next cell you will see the output generated by the parameters:  

**Description searched** : "modern seasonal cuisine"  

**Cuisine Type** : "Italian Contemporary, Creative"  

**Facilities & Services** : "Air Conditioning, Car Park, Interesting Wine List"  

**Price Range** : "€€€"

In [16]:
search_ui.output

Output()

## 4. Visualizing the Most Relevant Restaurants

### 4.1 Geocode Locations

In 4.1 we use geopy to get the coordinates of the restaurants and save them in the .tsv file. 

If some data of these restaurants is not found using geopy, their name, postal code and country are saved in a second file "noDataRest.tsv" to be used in any (possible) subsequent processing to recover the missing data of these restaurants.

In [15]:
import os
import pandas as pd
import json
from bs4 import BeautifulSoup
import glob

# Directory where .html files are stored
output_dir = 'C:/Users/xavie/Documents/ADM_HW3'  ## TO DO ##

# List to store restaurant data
restaurants_data = []

# Function to extract restaurant information from HTML
def extract_restaurant_info(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

        # Extract information using HTML structure of the page
        restaurant_info = {}

        script_tag = soup.find("script", type="application/ld+json")

        if script_tag:
            json_content = json.loads(script_tag.string)

            restaurant_info['restaurantName'] = json_content['name']
            restaurant_info['region'] = json_content['address']['addressRegion']
            restaurant_info['latitude'] = json_content['latitude']
            restaurant_info['longitude'] = json_content['longitude']

    return restaurant_info

# Loop through all files in the directory and extract information
# Itera attraverso le pagine da 1 a 100
for page_num in range(1, 101):
    # Costruisci il percorso della cartella per ciascuna pagina
    folder_path = os.path.join(output_dir, f"page_{page_num}")
    
    # Trova tutti i file di testo che iniziano con "html_" nella cartella corrente
    html_files = glob.glob(os.path.join(folder_path, "html_*.txt"))
    
    # Itera attraverso i file trovati e processa ciascuno
    for file_path in html_files:
        print(file_path)
        restaurant_info = extract_restaurant_info(file_path)
        restaurants_data.append(restaurant_info)

# Create a pandas DataFrame
df = pd.DataFrame(restaurants_data)

# Translation of italian region from english to italian
translations = {
    "Aosta Valley": "Valle d'Aosta/Vallée d'Aoste",
    "Piedmont": "Piemonte",
    "Lombardy": "Lombardia",
    "Sicily": "Sicilia",
    "Tuscany": "Toscana",
    "Apulia": "Puglia",
    "Trentino-South Tyrol": "Trentino-Alto Adige/Südtirol",
    "Sardinia": "Sardegna"
}
df['region'] = df['region'].replace(translations) 

# Save the data to a CSV file
df.to_csv("geodata.tsv", sep='\t', index=False)
print("Data saved to restaurants_data.csv")

To merge the information about the restaurants and their geo-localition info we need to merge the two different table: ```restaurants_data.tsv``` and ```geodata.tsv```.
To do this we use pandas command: ```merge```, but first, to avoid duplicates and bias we defined an attribute of join.
It need to be unique for each row of the two dataset. So we defined *restaurantId* that is an autoincrement value for each row of the dataset (primarey key).


In [5]:
df['restaurantId'] = range(1, len(df) + 1)
df.head()

,restaurantName,city,postalCode,country,address,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,restaurantId
0,Arca,San Benedetto del Tronto,63074,Italy,viale Rinascimento 137,€€€,"Modern Cuisine, Cuisine from Abruzzo","In business for over 20 years, Arca is definit...","['Air conditioning', 'Restaurant offering vege...","['Amex', 'Mastercard', 'Visa']",+39 0735 488908,https://www.arcaristorante.it/,1
1,Cannavacciuolo Le Cattedrali Asti,Asti,14100,Italy,frazione Valleandona 1/b,€€€€,Creative,"Situated in the hills outside Asti, this moder...","['Air conditioning', 'Car park', 'Garden or pa...","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 0141 185 8888,https://www.lecattedrali.com/,2
2,Capriccio,Manerba del Garda,25080,Italy,piazza San Bernardo 6 località Montinelle,€€€,"Modern Cuisine, Seafood",This restaurant standing on the shores of Lake...,"['Air conditioning', 'Car park', 'Great view',...","['Amex', 'Maestrocard', 'Mastercard', 'Visa']",+39 0365 551124,https://ristorantecapriccio.it/,3
3,Cestello Firenze,Florence,50124,Italy,piazza di Cestello 8,€€€,"Seafood, Contemporary",Situated in the beautiful piazza overlooked by...,"['Air conditioning', 'Terrace']","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 055 264 5364,NaN,4
4,Corte Sconta,Venice,30122,Italy,calle del Pestrin sestiere Castello 3886,€€€,"Seafood, Venetian","From its flooring to its tables, this restaura...","['Air conditioning', 'Terrace']","['Mastercard', 'Visa']",+39 041 522 7024,https://www.cortescontave.com,5


In [6]:
df.shape

(2006, 13)

We repeat the creation of the *restaurantId* column for the ```geodata.tsv```

In [7]:
df_geo = pd.read_csv("geodata.tsv", sep='\t')
df_geo['restaurantId'] = range(1, len(df) + 1)
df_geo.head()

,restaurantName,region,latitude,longitude,restaurantId
0,Arca,Marche,42.926408,13.897645,1
1,Cannavacciuolo Le Cattedrali Asti,Piemonte,44.907837,8.117304,2
2,Capriccio,Lombardia,45.552850,10.561210,3
3,Cestello Firenze,Toscana,43.770306,11.243657,4
4,Corte Sconta,Veneto,45.434791,12.347919,5


In [8]:
df_geo.shape

(2006, 5)

Now we can merge this two table, on the specified attribute (*restaurantId*), obtaining a new pandas DataFrame. 

In [9]:
df_final = pd.merge(df, df_geo, on='restaurantId')
print(df_final.shape)

(2006, 17)


In [10]:
df_final.head()

,restaurantName_x,city,postalCode,country,address,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,restaurantId,restaurantName_y,region,latitude,longitude
0,Arca,San Benedetto del Tronto,63074,Italy,viale Rinascimento 137,€€€,"Modern Cuisine, Cuisine from Abruzzo","In business for over 20 years, Arca is definit...","['Air conditioning', 'Restaurant offering vege...","['Amex', 'Mastercard', 'Visa']",+39 0735 488908,https://www.arcaristorante.it/,1,Arca,Marche,42.926408,13.897645
1,Cannavacciuolo Le Cattedrali Asti,Asti,14100,Italy,frazione Valleandona 1/b,€€€€,Creative,"Situated in the hills outside Asti, this moder...","['Air conditioning', 'Car park', 'Garden or pa...","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 0141 185 8888,https://www.lecattedrali.com/,2,Cannavacciuolo Le Cattedrali Asti,Piemonte,44.907837,8.117304
2,Capriccio,Manerba del Garda,25080,Italy,piazza San Bernardo 6 località Montinelle,€€€,"Modern Cuisine, Seafood",This restaurant standing on the shores of Lake...,"['Air conditioning', 'Car park', 'Great view',...","['Amex', 'Maestrocard', 'Mastercard', 'Visa']",+39 0365 551124,https://ristorantecapriccio.it/,3,Capriccio,Lombardia,45.552850,10.561210
3,Cestello Firenze,Florence,50124,Italy,piazza di Cestello 8,€€€,"Seafood, Contemporary",Situated in the beautiful piazza overlooked by...,"['Air conditioning', 'Terrace']","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 055 264 5364,NaN,4,Cestello Firenze,Toscana,43.770306,11.243657
4,Corte Sconta,Venice,30122,Italy,calle del Pestrin sestiere Castello 3886,€€€,"Seafood, Venetian","From its flooring to its tables, this restaura...","['Air conditioning', 'Terrace']","['Mastercard', 'Visa']",+39 041 522 7024,https://www.cortescontave.com,5,Corte Sconta,Veneto,45.434791,12.347919


As we can see from the output above the column restaurantName seems to be not unique, that's way we didn't use this column as attribute of join.
This is probably caused by the data gathering process, in which is possibile that some character differs from the two tables.
So, we decided to check how many rows differs in the relative column, then we drop one of the two and we keep it as "restaurantName".

In [11]:
df_final = df_final.rename(columns={'restaurantName_x': 'restaurantName'})

# Drop column 'restaurantName_y'
df_final = df_final.drop(columns=['restaurantName_y'])

Now we can see how is our ```df_final``` composed. We have all the information about the restaurant and we also added the information about its region and latidute and longitude.
This allow us to represent the restaurant and all its information as point in our map plot.

In [12]:
df_final.head()

,restaurantName,city,postalCode,country,address,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,restaurantId,region,latitude,longitude
0,Arca,San Benedetto del Tronto,63074,Italy,viale Rinascimento 137,€€€,"Modern Cuisine, Cuisine from Abruzzo","In business for over 20 years, Arca is definit...","['Air conditioning', 'Restaurant offering vege...","['Amex', 'Mastercard', 'Visa']",+39 0735 488908,https://www.arcaristorante.it/,1,Marche,42.926408,13.897645
1,Cannavacciuolo Le Cattedrali Asti,Asti,14100,Italy,frazione Valleandona 1/b,€€€€,Creative,"Situated in the hills outside Asti, this moder...","['Air conditioning', 'Car park', 'Garden or pa...","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 0141 185 8888,https://www.lecattedrali.com/,2,Piemonte,44.907837,8.117304
2,Capriccio,Manerba del Garda,25080,Italy,piazza San Bernardo 6 località Montinelle,€€€,"Modern Cuisine, Seafood",This restaurant standing on the shores of Lake...,"['Air conditioning', 'Car park', 'Great view',...","['Amex', 'Maestrocard', 'Mastercard', 'Visa']",+39 0365 551124,https://ristorantecapriccio.it/,3,Lombardia,45.552850,10.561210
3,Cestello Firenze,Florence,50124,Italy,piazza di Cestello 8,€€€,"Seafood, Contemporary",Situated in the beautiful piazza overlooked by...,"['Air conditioning', 'Terrace']","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 055 264 5364,NaN,4,Toscana,43.770306,11.243657
4,Corte Sconta,Venice,30122,Italy,calle del Pestrin sestiere Castello 3886,€€€,"Seafood, Venetian","From its flooring to its tables, this restaura...","['Air conditioning', 'Terrace']","['Mastercard', 'Visa']",+39 041 522 7024,https://www.cortescontave.com,5,Veneto,45.434791,12.347919


In [13]:
df_final.shape

(2006, 16)

### 4.3 Map Setup

Dash used instead of plain Plotly to make the map interactive and to be able to select regions with a simple click instead of using a list.

In [28]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import geopandas as gpd
import pandas as pd
import json

# DEBUG
print(df_final.shape)

# Load GeoJSON file of Italian regions in a GeoDataFrame
gdf = gpd.read_file("./GeoJson/limits_IT_regions.geojson")

# Check CRS and convert it if needed (important to calculate centroid of each region to display it centered)
if gdf.crs != "EPSG:4326":
    gdf = gdf.to_crs("EPSG:4326")

# Extract region names from the GeoDataFrame
region_names = gdf['reg_name'].tolist()

# Create Dash app (Dash provides an easy-to-use API for creating web apps using Python)
app = dash.Dash(__name__)

# Layout of the app with checkbox for filtering top_k_result.tsv
app.layout = html.Div([
    html.Div([
        # Add checkbox to filter top_k_result.tsv
        dcc.Checklist(
            id='show-top-k',
            options=[{'label': 'Show Top-K Restaurants Only', 'value': 'top_k'}],
            value=[],
            inline=True
        ),
    ]),
    
    # Create 2 maps, one of Italy with selectable region and one for the selected region where we display restaurants
    html.Div([
        dcc.Graph(id='italy-map', clickData=None, style={'width': '50vw', 'height': '100vh'}),
        dcc.Graph(id='region-map', style={'width': '50vw', 'height': '100vh'})
    ], style={'display': 'flex', 'flex-direction': 'row'})
])

# Global variable to store filtered restaurants (default is all restaurants)
filtered_restaurants = df_final

@app.callback(
    [Output('italy-map', 'figure'),
     Output('region-map', 'figure')],
    [Input('italy-map', 'clickData'),
     Input('show-top-k', 'value')]
)
def update_maps(clickData, show_top_k):
    global filtered_restaurants
    
    # Se la checkbox "Top-K" è selezionata, filtra i ristoranti dal file top_k_result.tsv
    if 'top_k' in show_top_k:
        top_k_df = pd.read_csv('top_k_result.tsv', sep='\t')
        filtered_restaurants = df_final[df_final['restaurantName'].isin(top_k_df['restaurantName'])]
    else:
        filtered_restaurants = df_final

    # Variabile per gestire se una regione è selezionata
    selected_region = None
    if clickData:
        selected_region = clickData['points'][0]['location']

    # Configurazione della mappa dell'Italia
    italy_map = px.choropleth_mapbox(
        geojson=json.loads(gdf.to_json()),
        locations=gdf['reg_name'],
        featureidkey="properties.reg_name",
        color_continuous_scale=['grey'],
        opacity=0.2,
        mapbox_style="carto-positron",
        zoom=4.5,
        center={"lat": 41.8719, "lon": 12.5674},
        title="Map of the Italian Regions"
    )
    
    # Se la checkbox "Top-K" è selezionata e una regione è selezionata, mostra i marker
    if 'top_k' in show_top_k and selected_region:
        for price_range, color in [('€', 'blue'), ('€€', 'green'), ('€€€', 'purple'), ('€€€€', 'brown')]:
            price_restaurants = filtered_restaurants[filtered_restaurants['priceRange'] == price_range]
            italy_map.add_scattermapbox(
                lat=price_restaurants['latitude'].tolist(),
                lon=price_restaurants['longitude'].tolist(),
                mode='markers',
                marker=dict(size=10, color=color, opacity=0.8),
                text=price_restaurants['restaurantName'],
                name=price_range,
                hovertemplate=(
                    "<b>Restaurant Name:</b> %{text}<br>" +
                    "<b>Latitude:</b> %{lat}<br>" +
                    "<b>Longitude:</b> %{lon}<br>"
                )
            )
    
    if selected_region:
        italy_map.update_traces(marker_line_color="red", selector=dict(location=selected_region))
    
    # Configurazione della mappa della regione selezionata (se presente)
    if selected_region:
        region_gdf = gdf[gdf['reg_name'] == selected_region]
        region_gdf_proj = region_gdf.to_crs(epsg=32632)
        centroid_proj = region_gdf_proj.geometry.centroid.iloc[0]
        centroid = gpd.GeoSeries([centroid_proj], crs="EPSG:32632").to_crs("EPSG:4326").iloc[0]
        center_lat, center_lon = centroid.y, centroid.x

        filtered_restaurants_region = filtered_restaurants[filtered_restaurants['region'] == selected_region]

        region_map = px.choropleth_mapbox(
            geojson=json.loads(region_gdf.to_json()),
            locations=[selected_region],
            featureidkey="properties.reg_name",
            mapbox_style="carto-positron",
            color_discrete_sequence=['red'],
            opacity=0.1,
            zoom=6,
            center={"lat": center_lat, "lon": center_lon},
            title=f"Map of {selected_region} with Restaurants"
        )
        
        # Aggiungi marker per i ristoranti nella regione selezionata
        for price_range, color in [('€', 'blue'), ('€€', 'green'), ('€€€', 'purple'), ('€€€€', 'brown')]:
            price_restaurants = filtered_restaurants_region[filtered_restaurants_region['priceRange'] == price_range]
            region_map.add_scattermapbox(
                lat=price_restaurants['latitude'].tolist(),
                lon=price_restaurants['longitude'].tolist(),
                mode='markers',
                marker=dict(size=10, color=color, opacity=0.8),
                text=price_restaurants['restaurantName'],
                name=price_range,
                hovertemplate=(
                    "<b>Restaurant Name:</b> %{text}<br>" +
                    "<b>Latitude:</b> %{lat}<br>" +
                    "<b>Longitude:</b> %{lon}<br>"
                )
            )
    else:
        # Se nessuna regione è selezionata, restituisci una mappa vuota
        region_map = {}

    return italy_map, region_map

# Start Dash App
if __name__ == '__main__':
    app.run_server(jupyter_mode = 'external', debug=True)


(2006, 16)
Dash app running on http://127.0.0.1:8050/


## 5. BONUS: Advanced Search Engine

In [39]:
def get_weighted_similarity(df, queries, fields):
    '''
    Compute the cosine similarity score weighted basing on the non empty fields
    '''
    # Determina i campi attivi e i pesi
    active_fields = [field for field in fields if queries[field].strip()]
    
    if not active_fields:
        return np.zeros(len(df))
    
    # Compute the weight structure basing on the active text fields
    if len(active_fields) == 1:
        weights = {field: 1.0 if field == active_fields[0] else 0.0 for field in fields}
    elif len(active_fields) == 2 and "restaurantName" in active_fields:
        weights = {
            "restaurantName": 0.7,
            "city": 0.3 if "city" in active_fields else 0.0,
            "cuisineType": 0.3 if "cuisineType" in active_fields else 0.0
        }
    elif len(active_fields) == 2 and "restaurantName" not in active_fields:
        weights = {
            "restaurantName": 0.0,
            "city": 0.7,
            "cuisineType": 0.3
        }
    elif len(active_fields) == 3:
        weights = {
            "restaurantName": 0.5,
            "city": 0.3,
            "cuisineType": 0.2
        }
    else:
        weights = {}
    
    # Compute the cosine similarity for each text with value
    field_similarities = defaultdict(lambda: np.zeros(len(df)))
    for field in active_fields:
        similarity_scores = engine.top_k_cosine_similarity(df[field], queries[field], k = 100) # RIMUOVERE K (ANCHE DALLA DESCRIZIONE)
        for doc_id, score in similarity_scores:
            field_similarities[field][doc_id] = score * weights[field]
    
    # Aggregate the scores
    total_score = np.zeros(len(df))
    for field in active_fields:
        total_score += field_similarities[field]
    
    return total_score / len(active_fields)


def get_price_range_value(price_str):
    '''
    Convert the € string in the number of correspondent elements
    '''
    return len(price_str.strip())

def search_restaurants(restaurant_name_widget, city_widget, cuisine_type_widget, 
                      price_range_widget, region_checkboxes,
                      credit_card_checkboxes, facility_checkboxes, output):
    with output:
        clear_output()
        
        try:
            # Raccolta input testuali
            queries = {
                "restaurantName": restaurant_name_widget.value.strip().lower(),
                "city": city_widget.value.strip().lower(),
                "cuisineType": cuisine_type_widget.value.strip().lower()
            }
            print("Query raccolte:", queries)  # Debugging

            # Calcola similarity score pesata
            similarity_scores = get_weighted_similarity(
                df_final,
                queries,
                ["restaurantName", "city", "cuisineType"]
            )
            print("Punteggi di similarità calcolati:", similarity_scores)  # Debugging
            
            # Crea un DataFrame temporaneo con gli scores
            results = df_final.copy()
            results['similarity_score'] = similarity_scores
            
            # Applica threshold per la similarity
            if any(queries.values()):
                results = results[results['similarity_score'] > 0.0]
            print("Risultati dopo filtro similarity > 0:", results.shape)  # Debugging
            
            # Applica filtri (aggiungi stampa per ogni filtro applicato)
            selected_price_range = price_range_widget.value
            selected_regions = [rc.description for rc in region_checkboxes if rc.value]
            selected_credit_cards = [ccc.description for ccc in credit_card_checkboxes if ccc.value]
            selected_facilities = [fc.description for fc in facility_checkboxes if fc.value]

            # Filtra per intervallo di prezzo
            if selected_price_range:
                results = results[
                    results["priceRange"].apply(get_price_range_value).between(
                        selected_price_range[0], 
                        selected_price_range[1]
                    )
                ]
            print("Risultati dopo filtro prezzo:", results.shape)  # Debugging

            if selected_regions:
                results = results[results["region"].isin(selected_regions)]
            print("Risultati dopo filtro regione:", results.shape)  # Debugging

            if selected_credit_cards:
                results = results[
                    results["creditCards"].apply(
                        lambda x: all(card in eval(x) for card in selected_credit_cards)
                    )
                ]
            print("Risultati dopo filtro carte di credito:", results.shape)  # Debugging

            if selected_facilities:
                results = results[
                    results["facilitiesServices"].apply(
                        lambda x: all(service in eval(x) for service in selected_facilities)
                    )
                ]
            print("Risultati dopo filtro servizi e strutture:", results.shape)  # Debugging

            # Ordina per similarity score
            if any(queries.values()):
                results = results.sort_values('similarity_score', ascending=False)

            # Mostra i risultati
            if not results.empty:
                display(results[["restaurantName", "address", "cuisineType", "priceRange", "website"]])
            else:
                print("Nessun risultato trovato.")
                
        except Exception as e:
            print(f"Errore: {e}")


# Collect all the possible values for Credit Cards, Services and facilities, regions
unique_credit_cards = sorted(list(set(chain(*df_final['creditCards'].apply(eval)))))
unique_services_facilities = sorted(list(set(chain(*df_final['facilitiesServices'].apply(eval)))))
regions = sorted(pd.Series(df_final["region"]).dropna().drop_duplicates().tolist())

# Widget input
restaurant_name = ipw.Text(placeholder="Type restaurant name", description="Restaurant:")
city = ipw.Text(placeholder="Type city", description="City:")
cuisine_type = ipw.Text(placeholder="Type cuisine", description="Cuisine:")
price_range = ipw.IntRangeSlider(
    value=[1, 4],
    min=1,
    max=4,
    step=1,
    description="Price Range:",
)

# Checkbox for regions, CreditCards, Facilities & Services
region_checkboxes = [ipw.Checkbox(value=False, description=option) for option in regions]
region_box = ipw.VBox(region_checkboxes)

credit_card_checkboxes = [ipw.Checkbox(value=False, description=option) for option in unique_credit_cards]
credit_card_box = ipw.VBox(credit_card_checkboxes)

facility_checkboxes = [ipw.Checkbox(value=False, description=option) for option in unique_services_facilities]
facility_box = ipw.VBox(facility_checkboxes)

# Search button
search_button = ipw.Button(description="Search", button_style="success")
output = ipw.Output()

def on_search_button_click(b):
    search_restaurants(
        restaurant_name, city, cuisine_type, price_range, region_checkboxes,
        credit_card_checkboxes, facility_checkboxes, output
    )
search_button.on_click(on_search_button_click)
# Widget display 
display(restaurant_name, city, cuisine_type, price_range)
display(ipw.Label("Regions:"))
display(region_box)
display(ipw.Label("Credit Cards:"))
display(credit_card_box)
display(ipw.Label("Services and Facilities:"))
display(facility_box)
display(search_button, output)

Bozza

In [ ]:
from IPython.display import clear_output
def get_weighted_similarity(df, queries, fields):
    """
    Calcola la similarity score pesata basata sui campi compilati
    """
    # Determina i campi attivi e i pesi
    active_fields = [field for field in fields if queries[field].strip()]
    
    if not active_fields:
        return np.zeros(len(df))
    
    # Calcola i pesi basati sui campi attivi
    if len(active_fields) == 1:
        weights = {field: 1.0 if field == active_fields[0] else 0.0 for field in fields}
    elif len(active_fields) == 2 and "restaurantName" in active_fields:
        weights = {
            "restaurantName": 0.7,
            "city": 0.3 if "city" in active_fields else 0.0,
            "cuisineType": 0.3 if "cuisineType" in active_fields else 0.0
        }
    elif len(active_fields) == 2 and "restaurantName" not in active_fields:
        weights = {
            "restaurantName": 0.0,
            "city": 0.7,
            "cuisineType": 0.3
        }
    elif len(active_fields) == 3:
        weights = {
            "restaurantName": 0.5,
            "city": 0.3,
            "cuisineType": 0.2
        }
    else:
        weights = {}
    
    # Calcola similarity per ogni campo attivo
    field_similarities = defaultdict(lambda: np.zeros(len(df)))
    for field in active_fields:
        similarity_scores = engine.top_k_cosine_similarity(df[field], queries[field], k = 100)
        for doc_id, score in similarity_scores:
            field_similarities[field][doc_id] = score * weights[field]
    
    # Aggrega gli scores
    total_score = np.zeros(len(df))
    for field in active_fields:
        total_score += field_similarities[field]
    
    return total_score / len(active_fields)


def get_price_range_value(price_str):
    """Converte la stringa di € nel numero corrispondente"""
    return len(price_str.strip())

def search_restaurants(restaurant_name_widget, city_widget, cuisine_type_widget, 
                      price_range_widget, region_checkboxes,
                      credit_card_checkboxes, facility_checkboxes, output):
    with output:
        clear_output()
        print("Avviando ricerca...")
        
        try:
            # Raccolta input testuali
            queries = {
                "restaurantName": restaurant_name_widget.value.strip().lower(),
                "city": city_widget.value.strip().lower(),
                "cuisineType": cuisine_type_widget.value.strip().lower()
            }
            print("Query raccolte:", queries)  # Debugging

            # Calcola similarity score pesata
            similarity_scores = get_weighted_similarity(
                df_final,
                queries,
                ["restaurantName", "city", "cuisineType"]
            )
            print("Punteggi di similarità calcolati:", similarity_scores)  # Debugging
            
            # Crea un DataFrame temporaneo con gli scores
            results = df_final.copy()
            results['similarity_score'] = similarity_scores
            
            # Applica threshold per la similarity
            if any(queries.values()):
                results = results[results['similarity_score'] > 0.0]
            print("Risultati dopo filtro similarity > 0:", results.shape)  # Debugging
            
            # Applica filtri (aggiungi stampa per ogni filtro applicato)
            selected_price_range = price_range_widget.value
            selected_regions = [rc.description for rc in region_checkboxes if rc.value]
            selected_credit_cards = [ccc.description for ccc in credit_card_checkboxes if ccc.value]
            selected_facilities = [fc.description for fc in facility_checkboxes if fc.value]

            # Filtra per intervallo di prezzo
            if selected_price_range:
                results = results[
                    results["priceRange"].apply(get_price_range_value).between(
                        selected_price_range[0], 
                        selected_price_range[1]
                    )
                ]
            print("Risultati dopo filtro prezzo:", results.shape)  # Debugging

            if selected_regions:
                results = results[results["region"].isin(selected_regions)]
            print("Risultati dopo filtro regione:", results.shape)  # Debugging

            if selected_credit_cards:
                results = results[
                    results["creditCards"].apply(
                        lambda x: all(card in eval(x) for card in selected_credit_cards)
                    )
                ]
            print("Risultati dopo filtro carte di credito:", results.shape)  # Debugging

            if selected_facilities:
                results = results[
                    results["facilitiesServices"].apply(
                        lambda x: all(service in eval(x) for service in selected_facilities)
                    )
                ]
            print("Risultati dopo filtro servizi e strutture:", results.shape)  # Debugging

            # Ordina per similarity score
            if any(queries.values()):
                results = results.sort_values('similarity_score', ascending=False)

            # Mostra i risultati
            if not results.empty:
                display(results[["restaurantName", "address", "cuisineType", "priceRange", "website"]])
            else:
                print("Nessun risultato trovato.")
                
        except Exception as e:
            print(f"Errore: {e}")


# Widget inizializzazione e display
# Preparazione dei dati
unique_credit_cards = sorted(list(set(chain(*df_final['creditCards'].apply(eval)))))
unique_services_facilities = sorted(list(set(chain(*df_final['facilitiesServices'].apply(eval)))))
regions = sorted(pd.Series(df_final["region"]).dropna().drop_duplicates().tolist())

# Widget input
restaurant_name = ipw.Text(placeholder="Type restaurant name", description="Restaurant:")
city = ipw.Text(placeholder="Type city", description="City:")
cuisine_type = ipw.Text(placeholder="Type cuisine", description="Cuisine:")
price_range = ipw.IntRangeSlider(
    value=[1, 4],
    min=1,
    max=4,
    step=1,
    description="Price Range:",
)

region_checkboxes = [ipw.Checkbox(value=False, description=option) for option in regions]
region_box = ipw.VBox(region_checkboxes)

credit_card_checkboxes = [ipw.Checkbox(value=False, description=option) for option in unique_credit_cards]
credit_card_box = ipw.VBox(credit_card_checkboxes)

facility_checkboxes = [ipw.Checkbox(value=False, description=option) for option in unique_services_facilities]
facility_box = ipw.VBox(facility_checkboxes)

search_button = ipw.Button(description="Search", button_style="success")
output = ipw.Output()

def on_search_button_click(b):
    print("Pulsante cliccato!")
    search_restaurants(
        restaurant_name, city, cuisine_type, price_range, region_checkboxes,
        credit_card_checkboxes, facility_checkboxes, output
    )
search_button.on_click(on_search_button_click)
# Display dei widget
display(restaurant_name, city, cuisine_type, price_range)
display(ipw.Label("Regions:"))
display(region_box)
display(ipw.Label("Credit Cards:"))
display(credit_card_box)
display(ipw.Label("Services and Facilities:"))
display(facility_box)
display(search_button, output)

In [14]:
from itertools import chain
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display, clear_output

def create_inverted_index(documents):
    """Create an inverted index per i documenti"""
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    return vectorizer, tfidf_matrix

def calculate_similarity_score(query, vectorizer, tfidf_matrix):
    """Calcola il cosine similarity score tra la query e i documenti"""
    if not query.strip():
        return np.zeros(tfidf_matrix.shape[0])
    
    query_vector = vectorizer.transform([query])
    similarity_scores = cosine_similarity(query_vector, tfidf_matrix).flatten()
    return similarity_scores

def get_weighted_similarity(df, queries, fields):
    """
    Calcola la similarity score pesata basata sui campi compilati
    
    Args:
    df: DataFrame con i dati dei ristoranti
    queries: dict con le query per ogni campo
    fields: lista dei campi da considerare
    
    Returns:
    array con gli scores pesati
    """
    # Inizializza i vectorizer e le matrici TF-IDF per ogni campo
    field_data = {}
    for field in fields:
        preprocessed_documents = []
        for doc in df[field]:
            preprocessed_doc = engine.preprocessing(doc)
            preprocessed_documents.append(preprocessed_doc)
        vectorizer, tfidf_matrix = (preprocessed_documents)
        field_data[field] = {
            'vectorizer': vectorizer,
            'tfidf_matrix': tfidf_matrix,
            'score': None
        }
    
    # Calcola i similarity scores per i campi compilati
    active_fields = [field for field in fields if queries[field].strip()]
    
    if not active_fields:
        return np.zeros(len(df))
    
    # Compute the weight structure basing on the active text fields
    if len(active_fields) == 1:
        weights = {field: 1.0 if field == active_fields[0] else 0.0 for field in fields}
    elif len(active_fields) == 2 and "restaurantName" in active_fields:
        weights = {
            "restaurantName": 0.7,
            "city": 0.3 if "city" in active_fields else 0.0,
            "cuisineType": 0.3 if "cuisineType" in active_fields else 0.0
        }
    elif len(active_fields) == 2 and "restaurantName" not in active_fields:
        weights = {
            "restaurantName": 0.0,
            "city": 0.7,
            "cuisineType": 0.3
        }
    else:
        weights = {
            "restaurantName": 0.5,
            "city": 0.3,
            "cuisineType": 0.2
        }
    
    # Compute the cosine similarity for each text with value
    for field in active_fields:
        score = calculate_similarity_score(
            queries[field],
            field_data[field]['vectorizer'],
            field_data[field]['tfidf_matrix']
        )
        field_data[field]['score'] = score * weights[field]
    
    # Aggrega gli scores
    total_score = sum(field_data[field]['score'] for field in active_fields)
    return total_score / len(active_fields)

# Collect all the possible values for Credit Cards, Services and facilities, regions and 
unique_credit_cards = sorted(list(set(chain(*df_final['creditCards'].apply(eval)))))
unique_services_facilities = sorted(list(set(chain(*df_final['facilitiesServices'].apply(eval)))))
regions = sorted(pd.Series(df_final["region"]).dropna().drop_duplicates().tolist())
cuisine_types = sorted(df_final['cuisineType'].unique().tolist())

# Widget input
restaurant_name = widgets.Text(placeholder="Type restaurant name", description="Restaurant:")
city = widgets.Text(placeholder="Type city", description="City:")
cuisine_type = widgets.Text(placeholder="Type cuisine", description="Cuisine:")
price_range = widgets.IntRangeSlider(
    value=[1, 4],
    min=1,
    max=4,
    step=1,
    description="Price Range:",
)

# Checkbox for regions, CreditCards, Facilities & Services
region_checkboxes = [widgets.Checkbox(value=False, description=option) for option in regions]
region_box = widgets.VBox(region_checkboxes)

credit_card_checkboxes = [widgets.Checkbox(value=False, description=option) for option in unique_credit_cards]
credit_card_box = widgets.VBox(credit_card_checkboxes)

facility_checkboxes = [widgets.Checkbox(value=False, description=option) for option in unique_services_facilities]
facility_box = widgets.VBox(facility_checkboxes)

# Pulsante cerca e output
search_button = widgets.Button(description="Search", button_style="success")
output = widgets.Output()

def get_price_range_value(price_str):
    """Convert the € string in the number of correspondent elements"""
    return len(price_str.strip())

def enanched_search_restaurants(restaurant_name_widget, city_widget, cuisine_type_widget, 
                      price_range_widget, region_checkboxes,
                      credit_card_checkboxes, facility_checkboxes, output):
    with output:
        clear_output()
        print("Searching restaurants...")
        
        try:
            # Take values of input queries
            queries = {
                "restaurantName": restaurant_name_widget.value.strip().lower(),
                "city": city_widget.value.strip().lower(),
                "cuisineType": cuisine_type_widget.value.strip().lower()
            }
            
            # Compute the weighted similarity score
            similarity_scores = get_weighted_similarity(
                df_final,
                queries,
                ["restaurantName", "city", "cuisineType"]
            )
            
            # Crea un DataFrame temporaneo con gli scores
            results = df_final.copy()
            results['similarity_score'] = similarity_scores
            
            # Show only the rows with similarity score > 0
            if any(queries.values()):
                results = results[results['similarity_score'] > 0.0]
            
            # Collect values of the filters
            selected_price_range = price_range_widget.value
            selected_regions = [rc.description for rc in region_checkboxes if rc.value]
            selected_credit_cards = [ccc.description for ccc in credit_card_checkboxes if ccc.value]
            selected_facilities = [fc.description for fc in facility_checkboxes if fc.value]

            # Apply the priceRange filter
            results = results[
                results["priceRange"].apply(get_price_range_value).between(
                    selected_price_range[0], 
                    selected_price_range[1]
                )
            ]

            # Apply the Region filter
            if selected_regions:
                results = results[results["region"].isin(selected_regions)]

            # Apply the CreditCards filter
            if selected_credit_cards:
                results = results[
                    results["creditCards"].apply(
                        lambda x: all(card in eval(x) for card in selected_credit_cards)
                    )
                ]

            # Apply the services & facilities filter
            if selected_facilities:
                results = results[
                    results["facilitiesServices"].apply(
                        lambda x: all(service in eval(x) for service in selected_facilities)
                    )
                ]

            # Order by similarity score
            if any(queries.values()):
                results = results.sort_values('similarity_score', ascending=False)

            # Show the results
            if not results.empty:
                display(results[["restaurantName", "address", "cuisineType", "priceRange", "website"]])
            else:
                print("No result founded.")
                
        except Exception as e:
            print(f"Error: {e}")

# Collegamento evento al pulsante
search_button.on_click(lambda b: enanched_search_restaurants(
    restaurant_name, city, cuisine_type, price_range, region_checkboxes,
    credit_card_checkboxes, facility_checkboxes, output
))

# Display the UI (widgets)
display(restaurant_name, city, cuisine_type, price_range)
display(widgets.Label("Regions:"))
display(region_box)
display(widgets.Label("Credit Cards:"))
display(credit_card_box)
display(widgets.Label("Services and Facilities:"))
display(facility_box)
display(search_button, output)

Text(value='', description='Restaurant:', placeholder='Type restaurant name')

Text(value='', description='City:', placeholder='Type city')

Text(value='', description='Cuisine:', placeholder='Type cuisine')

IntRangeSlider(value=(1, 4), description='Price Range:', max=4, min=1)

Label(value='Regions:')

Label(value='Credit Cards:')

Label(value='Services and Facilities:')

Button(button_style='success', description='Search', style=ButtonStyle())

Output()

In [21]:
import ipywidgets as widgets
def get_weighted_similarity(df, queries, fields):
    """
    Calcola la similarity score pesata basata sui campi compilati, utilizzando la funzione top_k_cosine_similarity.
    
    Args:
    df: DataFrame con i dati dei ristoranti
    queries: dict con le query per ogni campo
    fields: lista dei campi da considerare
    
    Returns:
    array con gli scores pesati
    """
    # Prepara i dati per ogni campo
    field_data = {}
    for field in fields:
        documents = df[field].fillna('').str.lower().tolist()
        field_data[field] = {
            'documents': documents,
            'score': None
        }

    # Campi attivi con query non vuote
    active_fields = [field for field in fields if queries[field].strip()]
    if not active_fields:
        return np.zeros(len(df))
    
    # Struttura dei pesi basata sui campi attivi
    if len(active_fields) == 1:
        weights = {field: 1.0 if field == active_fields[0] else 0.0 for field in fields}
    elif len(active_fields) == 2 and "restaurantName" in active_fields:
        weights = {
            "restaurantName": 0.7,
            "city": 0.3 if "city" in active_fields else 0.0,
            "cuisineType": 0.3 if "cuisineType" in active_fields else 0.0
        }
    elif len(active_fields) == 2 and "restaurantName" not in active_fields:
        weights = {
            "restaurantName": 0.0,
            "city": 0.7,
            "cuisineType": 0.3
        }
    else:
        weights = {
            "restaurantName": 0.5,
            "city": 0.3,
            "cuisineType": 0.2
        }
    
    # Calcolo della similarità usando top_k_cosine_similarity
    for field in active_fields:
        documents = field_data[field]['documents']
        query = queries[field].lower()
        similarities = engine.top_k_cosine_similarity(
            docs=pd.Series(documents),
            query=query,
            docs_preprocessed=False,
            query_preprocessed=False
        )
        print(f"Similarities for {field}: {similarities}")
        
        # Crea un array di score con 0 come default
        scores = np.zeros(len(documents))
        for doc_id, similarity in similarities:
            if similarity > 0:
                scores[doc_id] = similarity
        
        # Applica il peso per il campo
        field_data[field]['score'] = scores * weights[field]
    
    # Aggrega i punteggi
    total_score = sum(field_data[field]['score'] for field in active_fields)
    return total_score

unique_credit_cards = sorted(list(set(chain(*df_final['creditCards'].apply(eval)))))
unique_services_facilities = sorted(list(set(chain(*df_final['facilitiesServices'].apply(eval)))))
regions = sorted(pd.Series(df_final["region"]).dropna().drop_duplicates().tolist())

# Widget input
restaurant_name = widgets.Text(placeholder="Type restaurant name", description="Restaurant:")
city = widgets.Text(placeholder="Type city", description="City:")
cuisine_type = widgets.Text(placeholder="Type cuisine", description="Cuisine:")
price_range = widgets.IntRangeSlider(
    value=[1, 4],
    min=1,
    max=4,
    step=1,
    description="Price Range:",
)

# Checkbox for regions, CreditCards, Facilities & Services
region_checkboxes = [widgets.Checkbox(value=False, description=option) for option in regions]
region_box = widgets.VBox(region_checkboxes)

credit_card_checkboxes = [widgets.Checkbox(value=False, description=option) for option in unique_credit_cards]
credit_card_box = widgets.VBox(credit_card_checkboxes)

facility_checkboxes = [widgets.Checkbox(value=False, description=option) for option in unique_services_facilities]
facility_box = widgets.VBox(facility_checkboxes)

# Pulsante cerca e output
search_button = widgets.Button(description="Search", button_style="success")
output = widgets.Output()

def get_price_range_value(price_str):
    """Convert the € string in the number of correspondent elements"""
    return len(price_str.strip())

def enanched_search_restaurants(restaurant_name, city, cuisine_type, 
                                price_range, region_checkboxes,
                                credit_card_checkboxes, facility_checkboxes, output):
    with output:
        clear_output()
        print("Searching restaurants...")
        
        try:
            # Recupera i valori delle query
            queries = {
                "restaurantName": restaurant_name.value.strip().lower(),
                "city": city.value.strip().lower(),
                "cuisineType": cuisine_type.value.strip().lower()
            }
            
            # Calcola il punteggio di similarità pesato
            similarity_scores = get_weighted_similarity(
                df_final,
                queries,
                ["restaurantName", "city", "cuisineType"]
            )
            
            # Crea un DataFrame temporaneo con gli scores
            results = df_final.copy()
            results['similarity_score'] = similarity_scores
            
            # Filtra solo i risultati con score maggiore di 0
            results = results[results['similarity_score'] > 0.0]
            
            # Filtri aggiuntivi
            selected_price_range = price_range.value
            selected_regions = [rc.description for rc in region_checkboxes if rc.value]
            selected_credit_cards = [ccc.description for ccc in credit_card_checkboxes if ccc.value]
            selected_facilities = [fc.description for fc in facility_checkboxes if fc.value]

            # Filtro per intervallo di prezzo
            results = results[
                results["priceRange"].apply(get_price_range_value).between(
                    selected_price_range[0], 
                    selected_price_range[1]
                )
            ]

            # Filtro per regione
            if selected_regions:
                results = results[results["region"].isin(selected_regions)]

            # Filtro per carte di credito
            if selected_credit_cards:
                results = results[
                    results["creditCards"].apply(
                        lambda x: all(card in eval(x) for card in selected_credit_cards)
                    )
                ]

            # Filtro per servizi e strutture
            if selected_facilities:
                results = results[
                    results["facilitiesServices"].apply(
                        lambda x: all(service in eval(x) for service in selected_facilities)
                    )
                ]

            # Ordina i risultati per punteggio di similarità
            results = results.sort_values('similarity_score', ascending=False)

            # Mostra i risultati
            if not results.empty:
                display(results[["restaurantName", "address", "cuisineType", "priceRange", "website"]])
            else:
                print("No result founded.")
                
        except Exception as e:
            print(f"Error: {e}")
# Collegamento evento al pulsante
search_button.on_click(lambda b: enanched_search_restaurants(
    restaurant_name, city, cuisine_type, price_range, region_checkboxes,
    credit_card_checkboxes, facility_checkboxes, output
))

# Display the UI (widgets)
display(restaurant_name, city, cuisine_type, price_range)
display(widgets.Label("Regions:"))
display(region_box)
display(widgets.Label("Credit Cards:"))
display(credit_card_box)
display(widgets.Label("Services and Facilities:"))
display(facility_box)
display(search_button, output)


Text(value='', description='Restaurant:', placeholder='Type restaurant name')

Text(value='', description='City:', placeholder='Type city')

Text(value='', description='Cuisine:', placeholder='Type cuisine')

IntRangeSlider(value=(1, 4), description='Price Range:', max=4, min=1)

Label(value='Regions:')

Label(value='Credit Cards:')

Label(value='Services and Facilities:')

Button(button_style='success', description='Search', style=ButtonStyle())

Output()

In [21]:
def get_weighted_similarity(df, queries, fields):
    """
    Calcola la similarity score pesata basata sui campi compilati
    """
    # Determina i campi attivi e i pesi
    active_fields = [field for field in fields if queries[field].strip()]
    
    if not active_fields:
        return np.zeros(len(df))
    
    # Calcola i pesi basati sui campi attivi
    if len(active_fields) == 1:
        weights = {field: 1.0 if field == active_fields[0] else 0.0 for field in fields}
    elif len(active_fields) == 2 and "restaurantName" in active_fields:
        weights = {
            "restaurantName": 0.7,
            "city": 0.3 if "city" in active_fields else 0.0,
            "cuisineType": 0.3 if "cuisineType" in active_fields else 0.0
        }
    elif len(active_fields) == 2 and "restaurantName" not in active_fields:
        weights = {
            "restaurantName": 0.0,
            "city": 0.7,
            "cuisineType": 0.3
        }
    else:
        weights = {
            "restaurantName": 0.5,
            "city": 0.3,
            "cuisineType": 0.2
        }
    
    # Calcola similarity per ogni campo attivo
    field_similarities = defaultdict(lambda: np.zeros(len(df)))
    for field in active_fields:
        similarity_scores = engine.top_k_cosine_similarity(df[field], queries[field], k = 1000)
        for doc_id, score in similarity_scores:
            field_similarities[field][doc_id] = score * weights[field]
    
    # Aggrega gli scores
    total_score = np.zeros(len(df))
    for field in active_fields:
        total_score += field_similarities[field]
    
    return total_score / len(active_fields)


def get_price_range_value(price_str):
    """Converte la stringa di € nel numero corrispondente"""
    return len(price_str.strip())

def search_restaurants(restaurant_name_widget, city_widget, cuisine_type_widget, 
                      price_range_widget, region_checkboxes,
                      credit_card_checkboxes, facility_checkboxes, output):
    with output:
        clear_output()
        print("Avviando ricerca...")
        
        try:
            # Raccolta input testuali
            queries = {
                "restaurantName": restaurant_name_widget.value.strip().lower(),
                "city": city_widget.value.strip().lower(),
                "cuisineType": cuisine_type_widget.value.strip().lower()
            }
            print("Query raccolte:", queries)  # Debugging

            # Calcola similarity score pesata
            similarity_scores = get_weighted_similarity(
                df_final,
                queries,
                ["restaurantName", "city", "cuisineType"]
            )
            print("Punteggi di similarità calcolati:", similarity_scores)  # Debugging
            
            # Crea un DataFrame temporaneo con gli scores
            results = df_final.copy()
            results['similarity_score'] = similarity_scores
            
            # Applica threshold per la similarity
            if any(queries.values()):
                results = results[results['similarity_score'] > 0.0]
            print("Risultati dopo filtro similarity > 0:", results.shape)  # Debugging
            
            # Applica filtri (aggiungi stampa per ogni filtro applicato)
            selected_price_range = price_range_widget.value
            selected_regions = [rc.description for rc in region_checkboxes if rc.value]
            selected_credit_cards = [ccc.description for ccc in credit_card_checkboxes if ccc.value]
            selected_facilities = [fc.description for fc in facility_checkboxes if fc.value]

            # Filtra per intervallo di prezzo
            if selected_price_range:
                results = results[
                    results["priceRange"].apply(get_price_range_value).between(
                        selected_price_range[0], 
                        selected_price_range[1]
                    )
                ]
            print("Risultati dopo filtro prezzo:", results.shape)  # Debugging

            if selected_regions:
                results = results[results["region"].isin(selected_regions)]
            print("Risultati dopo filtro regione:", results.shape)  # Debugging

            if selected_credit_cards:
                results = results[
                    results["creditCards"].apply(
                        lambda x: all(card in eval(x) for card in selected_credit_cards)
                    )
                ]
            print("Risultati dopo filtro carte di credito:", results.shape)  # Debugging

            if selected_facilities:
                results = results[
                    results["facilitiesServices"].apply(
                        lambda x: all(service in eval(x) for service in selected_facilities)
                    )
                ]
            print("Risultati dopo filtro servizi e strutture:", results.shape)  # Debugging

            # Ordina per similarity score
            if any(queries.values()):
                results = results.sort_values('similarity_score', ascending=False)

            # Mostra i risultati
            if not results.empty:
                display(results[["restaurantName", "address", "cuisineType", "priceRange", "website"]])
            else:
                print("Nessun risultato trovato.")
                
        except Exception as e:
            print(f"Errore: {e}")


# Widget inizializzazione e display
# Preparazione dei dati
unique_credit_cards = sorted(list(set(chain(*df_final['creditCards'].apply(eval)))))
unique_services_facilities = sorted(list(set(chain(*df_final['facilitiesServices'].apply(eval)))))
regions = sorted(pd.Series(df_final["region"]).dropna().drop_duplicates().tolist())

# Widget input
restaurant_name = ipw.Text(placeholder="Type restaurant name", description="Restaurant:")
city = ipw.Text(placeholder="Type city", description="City:")
cuisine_type = ipw.Text(placeholder="Type cuisine", description="Cuisine:")
price_range = ipw.IntRangeSlider(
    value=[1, 4],
    min=1,
    max=4,
    step=1,
    description="Price Range:",
)

region_checkboxes = [ipw.Checkbox(value=False, description=option) for option in regions]
region_box = widgets.VBox(region_checkboxes)

credit_card_checkboxes = [ipw.Checkbox(value=False, description=option) for option in unique_credit_cards]
credit_card_box = widgets.VBox(credit_card_checkboxes)

facility_checkboxes = [ipw.Checkbox(value=False, description=option) for option in unique_services_facilities]
facility_box = widgets.VBox(facility_checkboxes)

search_button = ipw.Button(description="Search", button_style="success")
output = widgets.Output()

def on_search_button_click(b):
    print("Pulsante cliccato!")
    search_restaurants(
        restaurant_name, city, cuisine_type, price_range, region_checkboxes,
        credit_card_checkboxes, facility_checkboxes, output
    )
search_button.on_click(on_search_button_click)
# Display dei widget
display(restaurant_name, city, cuisine_type, price_range)
display(widgets.Label("Regions:"))
display(region_box)
display(widgets.Label("Credit Cards:"))
display(credit_card_box)
display(widgets.Label("Services and Facilities:"))
display(facility_box)
display(search_button, output)

Text(value='', description='Restaurant:', placeholder='Type restaurant name')

Text(value='', description='City:', placeholder='Type city')

Text(value='', description='Cuisine:', placeholder='Type cuisine')

IntRangeSlider(value=(1, 4), description='Price Range:', max=4, min=1)

Label(value='Regions:')

Label(value='Credit Cards:')

Label(value='Services and Facilities:')

Button(button_style='success', description='Search', style=ButtonStyle())

Output()

# Algorithmic Question (AQ)



### Pseudocode

The intuition is that, starting from the origin (0, 0) in the first quadrant where all packages are located, the first package is always reachable. Given that we can only move up or right, we can only reach packages that are above or to the right of the current one. Therefore, for each package reached, we need to check the next one: if it is below or to the left, we can print "NO" because it’s unreachable. Otherwise, we proceed, but to ensure the lexicographically smallest path, we need to sort all packages in ascending order by their coordinates.

Below is the pseudocode of an algorithm that solves this problem:

$ \mathbf{PackageCollector(t)} $

(0) $ \textbf{for text case t}: $

* (1) $ read(n) $

* (2) $ \text{packages} ← \text{[ ]} $

* (3) $ \textbf{for i=1 to n:} $ // $O(n)$

 * (4) $ read(x[i]), read(y[i]) $
 * (5) $ \text{packages} \leftarrow \text{packages} + [(x[i], y[i])] $
* (6) $ \textbf{Sort} \text{ packages by x and then y coordinate in ascending order}$ // $ O(n \log(n)) $
* (7) $ \text{x_current} \leftarrow 0 $
* (8) $ \text{y_current} \leftarrow 0 $
* (9) $ \text{path } \leftarrow \text{' '}$
* (10) $ \text{possible} \leftarrow \text{TRUE} $

* (11) $\textbf{for i=1 to n:} $ // $O(n)$

  * (12) $ \textbf{if } \mathbf{x[i]<}\textbf{x_current or } \mathbf{y[i]<}\textbf{y_current:} $
    * (13) $ \text{possible} \leftarrow \text{FALSE} $
    * (14) $ break $
  * $ \textbf{end if} $
  * (15) $ \text{num_right_moves} \leftarrow x[i]-\text{x_current} $
  * (16) $ \text{num_up_moves} \leftarrow y[i]-\text{y_current} $
  * (17) $ \text{path} \leftarrow \text{path} + \text{'R'} \cdot \text{num_right_moves} $
  * (18) $ \text{path} \leftarrow \text{path} + \text{'U'} \cdot \text{num_up_moves} $
  * (19) $ \text{x_current} \leftarrow x[i]$
  * (20) $ \text{y_current} \leftarrow y[i]$
* $ \textbf{end for} $
* (21) $ \textbf{if possible:} $
    * (22) $ output(\text{'YES'}) $
    * (23) $ output(\text{path}) $
* $ \textbf{end if} $
* (24) $ \textbf{else:} $
  * (25) $ output(\text{'NO'}) $

$ \textbf{end for} $



---



### Proof of Correctness

The algorithm aims to determine a path from the origin $(0,0)$ to each package coordinate $(x[i],y[i])$, using only right and up movements. We will show that the algorithm checks when such a path is possible, and, if it is, how to build a valid sequence of right ```R``` and up ```U``` movements to visit all the target locations in order.

The algorithm works on a set of inputs whose general structure is known. It first reads the number $n$ of packages (1), and then their coordinates (4).

Since the robot can move right and up, similar to a staircase, it is able to reach any package that is in the first quadrant of the moving reference system centered in the current robot's cell. To do this, the robot just needs to move the number of steps to the right that it takes to reach the package's x coordinate, and move up for a number of steps corresponding to the difference in y coordinates between robot and package.

Every time there is at least one package in the robot's top-right quadrant, the algorithm must make sure that the robot does not skip packages, if it is avoidable. Otherwise, it might erroneously conclude that there is no path to reach all the packages. The sorting (6) ensures that the sequence of coordinates is non-decreasing in the x- and y-axes, and the robot visits them in the natural order tailored to its freedom of movement.

Step (12) evaluates whether the next package in the sequence is reachable by the robot. If only one of the coordinates of the next package in line is smaller than the robot's current position coordinates, the algorithm assigns a ```False``` value to the variable ```possible``` and later prints 'NO' in (25).

If all package coordinates are reachable in the sorted order, the algorithm constructs the path as follows:
   - For each package coordinate $(x[i], y[i])$, add `R` repeated $ x[i] - \text{x_current} $ times to move to the right (15)
   - Then, add `U` repeated $ y[i] - \text{y_current} $ times to move upward (16)

This process continues for all packages until a complete path is generated from $(0, 0)$ to the final destination.

The algorithm correctly identifies and implements a path when it exists because of the sorting step (6), the validity check in (12) and the path construction steps in (15)-(18). These passages enable the robot to traverse the grid and collect packages without backtracking.

On the other hand, if a path through all the package coordinates does not exist, this fact will emerge from the check (12). It can happen when, for example, the next package in the sequence is close in x coordinate and high in the y coordinate, but a later package that is next in the x coordinate, has a lower y coordinate. This is because we prioritized horizontal coordinates over vertical ones. Thanks to (12), the algorithm will detect cases like these.

**Termination**: The loop iterates through all $n$ packages exactly once, and each reachable package updates ```x_current``` and ```y_current``` without backtracking. Therefore, the algorithm terminates after a finite umber of steps.

**Correct Path**: If the algorithm outputs 'YES' and a path, then all packages are reachable following the non-decreasing sequence of ```x``` and ```y``` coordinates. The resulting path ensures each package is visited in sorted order.

**Verifying Existence**: If a path crossing all the packages starting from $(0,0)$ and moving only right and up does not exist, the algorithm detects it in the validity check and outputs 'NO'.



---



### Algorithm Complexity

Let us look at the algorithm step by step and calculate its complexity.

* The algorithm starts with a loop over the test cases (0), so it is executed $t$ times. Therefore, the total running time of the algorithm will be the running time of the operations inside this for loop, times $t$
* Afterwards, the algorithm reads $n$, the number of packages (1), and initializes the variable $\text{packages}$ where the package coordinates will be stored (2). These two operations take constant time $O(1)$
* Next, we find a for loop (3) that is executed $n$ times, once for each package whose coordinates need to be read. Reading the coordinates $x[i]$ and $y[i]$ happens in (4), then the coordinates are added to the $\text{packages}$ array (5). Both (4) and (5) require constant time, and since the for loop is called $n$ times, for a single test case this segment will cost $O(n)$.
* Step (6) sorts the package locations by x and y coordinates. This means that the coordinates are primarily sorted by their first entry, and if two locations have the same x coordinate, they will be sorted by their second coordinate. In the worst case, sorting will take $O(2 \cdot n \log(n))$, which is equivalent to $O(n \log(n))$.
* Steps (7) and (8) initialize the coordinates of the robot's current location, which initially is $(0,0)$, in $O(1)$ time
* Step (9) initializes the $\text{path}$ string ($O(1)$) where the moves of the robot will be saved, and step (10) initializes a boolean variable $\text{possible}$ ($O(1)$), 'True' by default, which will be converted to 'False' in case it is not possible to construct the path through all the packages
* After this, we have another for loop (11) that iterates over the coordinates in the $packages$ array and is executed at most $n$ times.
* The for loop begins with an if loop (12) that checks if the coordinates $x[i]$ and $y[i]$ are out of reach for the robot from its current position. If they are reachable, the algorithm ignores the operations in the if cycle and continues to build the path. If the coordinates are not reachable, the $\text{possible}$ variable is set to 'False' in (13) ($O(1)$) and the algorithm exits the for loop over the packages (14) ($O(1)$).
* In case the coordinate validity check is positive, the algorithm continues to execute lines (15) to (20). Here, it first calculates how many steps the robot has to move right and up to reach the next package ((15) and (16), $O(1)$), then updates the $\text{path}$ variable to include the new right and up moves ((17) and (18), $O(1)$). Finally, the current position of the robot is set to the current package's coordinates ((19) and (20), $O(1)$). All these operations together require twelve steps, considering addition, multiplication and assignement, so they all take constant time $O(1)$.
* This concludes the for loop starting at (11), which executes constant operations at least $n$ times. Therefore, this loop costs $O(n)$ time in total.
* At the end, we have an if loop (21) that checks if the $\text{possible}$ variable is true. If yes, it prints 'YES' (22) and the previously computed $\text{path}$ (23), requiring constant time $O(1)$. In case $\text{possible}$ is False, an else block (24) is triggered, printing 'NO' (25). 

Putting all of this together, we get a worst-case total running time of $O(n\log(n) + n)$ for each test case. Since $n\log(n)$ is asymptotically stronger than $n$, we can rewrite this in big-O notation as $O(n\log(n))$. This is the complexity for one test case. Considering all $t$ test cases, we get $O(t \cdot n\log(n))$. The most expensive part is the sorting step at (6), and all other segments are at most linear.



---


### LLM Complexity Evaluation

Here is the complexity analysis of the algorithm provided by the LLM ChatGPT:

### Step-by-Step Analysis of Time Complexity

#### 1. Input Reading and Initialization (Steps 1-2):
- Reading $n$ takes $O(1)$.
- Initializing the `packages` list takes $O(1)$.

#### 2. Reading Package Coordinates (Steps 3-5):
- The loop runs $n$ times, and in each iteration:
  - Reading $x[i]$ and $y[i]$ takes $O(1)$.
  - Adding a tuple to the list takes $O(1)$.
- **Total for this loop**: $O(n)$.

#### 3. Sorting the Packages (Step 6):
- Sorting $n$ elements based on two keys takes $O(n \log n)$.

#### 4. Path Construction and Validation (Steps 11-20):
- The loop runs $n$ times, and in each iteration:
  - Comparing $x[i]$ and $y[i]$ with `x_current` and `y_current` takes $O(1)$ (Step 12).
  - Updating the `path` string involves concatenating strings for right and up moves. Let $k$ represent the total number of moves across all packages. Since $k \leq n$ in the worst case:
    - Adding characters to the string takes $O(k)$ in total.
  - Updating `x_current` and `y_current` takes $O(1)$.
- **Total for this loop**: $O(n)$.

#### 5. Output Result (Steps 21-25):
- Printing "YES" or "NO" and the path involves $O(n)$ for the path string output in the worst case.

### Per Test Case Complexity:
Summing the complexities for a single test case:
- Reading input: $O(n)$
- Sorting: $O(n \log n)$
- Path construction: $O(n)$
- Outputs: $O(n)$

Thus, the total complexity for a single test case is:
$ O(n + n \log n + n) = O(n \log n)$

### Accounting for $t$ Test Cases:
With $t$ test cases, the above steps are repeated $t$ times. Therefore, the total complexity is: $O(t \cdot n \log n)$

### Final Time Complexity:
$O(t \cdot n \log n)$

Both our analysis and the one provided by the LLM reach the same conclusion. The analysis is accurate because we are evaluating the worst-case running times in big-O notations for each test case, and then multiplying them by the number of test cases $t$. For each test case, we have: 

$O(1)$ (read $n$ and initialize $\text{packages}$) $+ O(n)$ (read $2\cdot n$ coordinates) $+O(n\log(n))$ (sort coordinates) $+O(n)$ (for loop over coordinates with internal constant time operations) $+O(1)$ (final if-else block to print results) $=O(n\log(n))$

For $t$ test cases, this becomes $O(t \cdot n\log(n))$


### Greedy Algorithm

A simplified pseudocode of a greedy algorithm that finds a path to collect all the packages is the following:

### $ \mathbf{GreedyPackageCollector(t)} $

(0) $ \textbf{for text case t} $:
    
* (1) $ read(n) $
* (2) $ \text{packages} \leftarrow \text{[ ]} $
* (3) $ \textbf{for i=1 to n}$: // $ O(n) $
    * (4) $ read(x[i]), \text{read}(y[i]) $
    * (5) $ \text{packages} \leftarrow \text{packages} + [(x[i], y[i])] $

* (6) $ \text{x_current} \leftarrow 0 $
* (7) $ \text{y_current} \leftarrow 0 $
* (8) $ \text{path} \leftarrow \text{''} $

* (9) $ \textbf{while packages} \neq \text{[ ]} $: // Continue until all packages are collected
  * (10) $ \text{min_distance} \leftarrow \infty $
  * (11) $ \text{closest_package} \leftarrow \text{None} $
  * (12) $ \textbf{for (x, y)  in packages} $: // $ O(k) $, where $ k $ is the number of remaining packages
    * (13) $ \text{distance} \leftarrow |x - \text{x_current}| + |y - \text{y_current}| $
    * (14) $ \textbf{if distance} < \textbf{min_distance} $:
      * (15) $ \text{min_distance} \leftarrow \text{distance} $
      * (16) $ \text{closest_package} \leftarrow (x, y) $
  * (17) $ (\text{x_closest}, \text{y_closest}) \leftarrow \text{closest_package} $
  * (18) $ \text{packages} \leftarrow \text{packages} ((\text{x_closest}, \text{y_closest})) $
  
  * (19) $ \text{num_right_moves} \leftarrow \max(0, \text{x_closest} - \text{x_current}) $
  * (20) $ \text{num_left_moves} \leftarrow \max(0, \text{x_current} - \text{x_closest}) $
  * (21) $ \text{num_up_moves} \leftarrow \max(0, \text{y_closest} - \text{y_current}) $
  * (22) $ \text{num_down_moves} \leftarrow \max(0, \text{y_current} - \text{y_closest}) $

  * (23) $ \text{path} \leftarrow \text{path} + \text{'R'} \cdot \text{num_right_moves} $
  * (24) $ \text{path} \leftarrow \text{path} + \text{'L'} \cdot \text{num_left_moves} $
  * (25) $ \text{path} \leftarrow \text{path} + \text{'U'} \cdot \text{num_up_moves} $
  * (26) $ \text{path} \leftarrow \text{path} + \text{'D'} \cdot \text{num_down_moves} $

  * (27) $ \text{x_current} \leftarrow \text{x_closest} $
  * (28) $ \text{y_current} \leftarrow \text{y_closest} $

* (29) $ output(\text{path}) $

$ \textbf{end for} $


This algorithm uses a greedy approach to move to the packages that are currently closest, minimizing the Manhattan distance at each step. It always calculates a path that includes all the packages because it can move in every direction, but the distance travelled is not always optimal.

We can consider this counterexample: suppose the robot starts at position $(0,0)$ and the packages are distributed in these cells: $p_1:(1,0)$, $p_2:(2,0)$, $p_3:(3,0)$, $p_4:(4,0)$, $p_5: (5,0)$, $p_6: (6,0)$, $p_7: (7,0)$ and $p_8: (0,2)$. Initially, the algorithms calculates the distances between the robot and each package, and finds that the closest package is $p_1$, so it moves to $(1,0)$. After this, the algorithm searches for the next closest neighbour, which is $p_2$. Following the algorithm's logic, the robot will continue to collect packages $p_3$ to $p_7$ in order, and finally $p_8$.

The total distance that the robot travels using the greedy approach in this scenario is 16 units (1 unit = length of a square in the grid), because it takes the path 'RRRRRRRLLLLLLLUU'. However, if the robot would have first visited package $p_2$, which initially was not the closest package, and then all the others in order, it would only have travelled a distance of 11 units along the path 'UUDDRRRRRRR'.

The plot below illustrates this specific case of the problem:

In [ ]:
import matplotlib.pyplot as plt

# Coordinates
packages = [(1, 0), (2, 0), (3, 0), (4,0), (5,0), (6,0), (7, 0), (0, 2)]
x_coords, y_coords = zip(*packages)
robot = (0,0)

# Plot points
plt.figure(figsize=(6, 6))
plt.scatter(x_coords, y_coords, color='blue', label='Points')
plt.scatter(robot[0], robot[1], color='red', label='Robot', marker='x')

# Add labels to points
for x, y in packages:
    plt.text(x + 0.1, y + 0.1, f"({x},{y})", fontsize=9)
plt.text(0.1, 0.1, "Robot", fontsize=9)

# Set plot limits for better visualization
plt.xlim(-1, max(x_coords) + 2)
plt.ylim(-1, max(y_coords) + 2)

# Adjust plot
plt.gca().set_aspect('equal', adjustable='box')

# Add grid and labels
plt.axhline(0, color='black', linewidth=0.5, linestyle='--')
plt.axvline(0, color='black', linewidth=0.5, linestyle='--')
plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.xticks(range(1, 9))
plt.title('Robot in the Warehouse')
plt.legend()
plt.show()


---

The following code implements the PackageCollector algorithm explained in the first three points:

In [ ]:
def PackageCollector(t):
    '''
    Function that determines if a path traversing all the packages exists, and if it does, it outputs that path
    Input:
    t = number of test cases
    Output:
    None
    '''
    for _ in range(t):  # Loop over each test case
        n = int(input())  # Read the number of packages
        packages = []

        # Read packages
        for i in range(n):
            x, y = map(int, input().split())  # Read coordinates of the i-th package
            packages.append((x, y))

        packages = sorted(packages)  # Sort packages by x and y coordinates

        # Initializations
        x_current = 0  # current x-coordinate (starting at (0, 0))
        y_current = 0  # current y-coordinate (starting at (0, 0))
        path = ""  # initialize path
        possible = True  # True by default, turns False if a path is not possible

        for i in range(n):
            x, y = packages[i]

            if x < x_current or y < y_current: # check if package is reacheable
                possible = False  # cannot reach this package if it's behind in x or y
                break

            # Calculate the number of right and up moves
            num_right_moves = x - x_current
            num_up_moves = y - y_current
            
            path += 'R' * num_right_moves # Add the right moves to the path
            path += 'U' * num_up_moves # Add the up moves to the path

            # Update the current position to the current package's coordinates
            x_current = x
            y_current = y

        # Output results based on whether the path is possible
        if possible:
            print("YES")
            print(path)
        else:
            print("NO")

Below, we test the algorithm on a specific set of inputs:

3

5

1 3

1 2

3 3

5 5

4 3

2

1 0

0 1

1

4 3

In [ ]:
t = int(input())  # Number of test cases
PackageCollector(t)